Bài tập thực hành python
Bài 1: 
Thực hiện lấy dữ liệu thời tiết từ url sau: https://www.pokemon.com/us/api/pokedex/kalos
Yêu cầu:
+ Lấy thông tin dữ liệu các trường: number, height, weight, name, type, ThumbnailImage và lưu vào một file .csv
+ Dựa vào dữ liệu đã lấy được đó. Hãy thực hiện lấy thông tin của các Pokemon có type là poison


In [1]:
import requests
import csv

# URL của PokéAPI để lấy danh sách Pokémon (vùng Kalos)
url = "https://pokeapi.co/api/v2/pokedex/12/"  # 12 là pokedex của vùng Kalos

# Hàm lấy dữ liệu từ API và lưu vào file CSV
def fetch_and_save_pokemon_data():
    try:
        # Gửi yêu cầu GET đến API
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Kiểm tra nếu yêu cầu không thành công
        data = response.json()  # Chuyển đổi dữ liệu JSON

        # Danh sách để lưu thông tin Pokémon
        pokemon_list = []

        # Lấy danh sách Pokémon từ pokedex
        for entry in data['pokemon_entries']:
            pokemon_url = entry['pokemon_species']['url']
            try:
                # Gửi yêu cầu để lấy chi tiết Pokémon
                pokemon_response = requests.get(pokemon_url.replace('pokemon-species', 'pokemon'), timeout=10)
                pokemon_response.raise_for_status()
                pokemon_data = pokemon_response.json()

                # Lấy các trường yêu cầu
                pokemon_info = {
                    'number': pokemon_data.get('id', ''),
                    'height': pokemon_data.get('height', ''),
                    'weight': pokemon_data.get('weight', ''),
                    'name': pokemon_data.get('name', '').capitalize(),
                    'type': ', '.join([t['type']['name'] for t in pokemon_data.get('types', [])]),
                    'ThumbnailImage': pokemon_data['sprites'].get('front_default', '')
                }
                pokemon_list.append(pokemon_info)

            except requests.exceptions.RequestException as e:
                print(f"Lỗi khi lấy dữ liệu Pokémon {entry['pokemon_species']['name']}: {e}")
                continue

        # Lưu dữ liệu vào file CSV
        csv_file = 'pokemon_data.csv'
        with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=['number', 'height', 'weight', 'name', 'type', 'ThumbnailImage'])
            writer.writeheader()
            for pokemon in pokemon_list:
                writer.writerow(pokemon)

        print(f"Dữ liệu đã được lưu vào {csv_file}")

        return pokemon_list

    except requests.exceptions.RequestException as e:
        print(f"Lỗi khi lấy dữ liệu từ API: {e}")
        return []
    except ValueError as e:
        print(f"Lỗi khi phân tích JSON: {e}")
        return []

# Hàm lọc Pokémon có type là Poison
def filter_poison_pokemon(pokemon_list):
    poison_pokemon = [pokemon for pokemon in pokemon_list if 'poison' in pokemon['type'].lower()]
    
    # In thông tin Pokémon có type Poison
    print("\nDanh sách Pokémon có type Poison:")
    for pokemon in poison_pokemon:
        print(f"Name: {pokemon['name']}, Number: {pokemon['number']}, Type: {pokemon['type']}, "
              f"Height: {pokemon['height']}, Weight: {pokemon['weight']}, Image: {pokemon['ThumbnailImage']}")

    # Lưu Pokémon có type Poison vào file CSV riêng
    csv_file = 'poison_pokemon.csv'
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['number', 'height', 'weight', 'name', 'type', 'ThumbnailImage'])
        writer.writeheader()
        for pokemon in poison_pokemon:
            writer.writerow(pokemon)

    print(f"\nDữ liệu Pokémon có type Poison đã được lưu vào {csv_file}")

# Thực thi chương trình
if __name__ == "__main__":
    # Lấy và lưu dữ liệu Pokémon
    pokemon_data = fetch_and_save_pokemon_data()
    
    # Lọc và lưu Pokémon có type Poison
    if pokemon_data:
        filter_poison_pokemon(pokemon_data)

Dữ liệu đã được lưu vào pokemon_data.csv

Danh sách Pokémon có type Poison:
Name: Weedle, Number: 13, Type: bug, poison, Height: 3, Weight: 32, Image: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/13.png
Name: Kakuna, Number: 14, Type: bug, poison, Height: 6, Weight: 100, Image: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/14.png
Name: Beedrill, Number: 15, Type: bug, poison, Height: 10, Weight: 295, Image: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/15.png
Name: Budew, Number: 406, Type: grass, poison, Height: 2, Weight: 12, Image: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/406.png
Name: Roselia, Number: 315, Type: grass, poison, Height: 3, Weight: 20, Image: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/315.png
Name: Roserade, Number: 407, Type: grass, poison, Height: 9, Weight: 145, Image: https://raw.githubusercontent.com/PokeAPI/sprites/maste

Bài 2:
Lấy dữ liệu từ file excel từ link: https://docs.google.com/spreadsheets/d/1BnOzoEG0s6c8MpiUANZ0_pawXNHqdkid/edit?usp=sharing&ouid=115874127894901285908&rtpof=true&sd=true

Hãy thực hiện lọc dữ liệu của các dữ liệu với điều kiện sau:
- Trường dữ liệu có cột vpv2 và pDisCharge có giá trị chẵn, cột prec lẻ lưu vào trong file mới có tên: Data_new.csv
- Thực hiện tính tổng dữ liệu của từng hàng vBus1, vBus2, tạo một cột mới có tên Sum_vBUS và ghi kết quả vào đó.



In [6]:
import pandas as pd
import requests
from io import BytesIO

# B1: Tải file từ Google Sheets
url = "https://docs.google.com/spreadsheets/d/1BnOzoEG0s6c8MpiUANZ0_pawXNHqdkid/export?format=xlsx"
response = requests.get(url)
df = pd.read_excel(BytesIO(response.content))

# B2: Chuyển các cột liên quan về kiểu số (nếu chưa)
cols_to_convert = ['vpv2', 'pDisCharge', 'prec', 'vBus1', 'vBus2']
for col in cols_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# B3: Lọc dữ liệu theo điều kiện
filtered_df = df[
    (df['vpv2'] % 2 == 0) &
    (df['pDisCharge'] % 2 == 0) &
    (df['prec'] % 2 == 1)
].copy()

# B4: Tính tổng vBus1 + vBus2 → cột Sum_vBUS
filtered_df['Sum_vBUS'] = filtered_df['vBus1'] + filtered_df['vBus2']

# B5: Ghi ra file CSV mới
filtered_df.to_csv("Data_new.csv", index=False, encoding='utf-8-sig')

print("✅ Đã lọc và ghi dữ liệu vào file 'Data_new.csv'")

✅ Đã lọc và ghi dữ liệu vào file 'Data_new.csv'


Tạo một chương trình quản lý nhà để xe, sử dụng OOP với các lớp như: Info_Xe, Money_Time.
(Info_Xe sẽ chứa một số trường thông tin như: Loại xe (Xe đạp, xe máy, xe điện, ô tô, ...), chủ xe, thời gian gửi xe (tính theo giờ), biển số xe (Nếu có). Money_Time bao gồm: Giá tiền của từng xe trong 1 giờ. Quy định: Xe đạp: 2k/giờ, Xe máy: 5k/giờ, Xe điện: 3.5k/giờ, ô tô: 10k/giờ)
Yêu cầu: 
- Hệ thống cho phép thêm, sửa, xóa thông tin, cập nhật thông tin của Info_Xe hoặc Money_Time.
- Tính toán giá thành gửi xe của từng người, xuất thông tin những người đã gửi xe trên 20k
- Ghi dữ liệu vào trong file excel

In [8]:
import pandas as pd

# Lớp Info_Xe chứa thông tin xe
class Info_Xe:
    def __init__(self, loai_xe, chu_xe, thoi_gian, bien_so_xe=None):
        self.loai_xe = loai_xe
        self.chu_xe = chu_xe
        self.thoi_gian = thoi_gian  # thời gian gửi xe tính theo giờ
        self.bien_so_xe = bien_so_xe

    def __str__(self):
        return f"Loại xe: {self.loai_xe}, Chủ xe: {self.chu_xe}, Thời gian gửi xe: {self.thoi_gian} giờ, Biển số: {self.bien_so_xe if self.bien_so_xe else 'Không có'}"

# Lớp Money_Time chứa thông tin về giá xe
class Money_Time:
    def __init__(self):
        # Định nghĩa giá tiền cho các loại xe
        self.gia_xe = {
            "Xe đạp": 2,
            "Xe máy": 5,
            "Xe điện": 3.5,
            "Ô tô": 10
        }
    
    def tinh_gia(self, loai_xe, thoi_gian):
        # Tính giá gửi xe theo loại xe và thời gian
        if loai_xe in self.gia_xe:
            return self.gia_xe[loai_xe] * thoi_gian
        return 0  # Nếu loại xe không hợp lệ, trả về 0

# Quản lý các xe gửi
class QuanLyXe:
    def __init__(self):
        self.danh_sach_xe = []
        self.money_time = Money_Time()

    def them_xe(self, xe):
        self.danh_sach_xe.append(xe)
    
    def sua_xe(self, bien_so_xe, loai_xe=None, chu_xe=None, thoi_gian=None):
        for xe in self.danh_sach_xe:
            if xe.bien_so_xe == bien_so_xe:
                if loai_xe: xe.loai_xe = loai_xe
                if chu_xe: xe.chu_xe = chu_xe
                if thoi_gian: xe.thoi_gian = thoi_gian
                break

    def xoa_xe(self, bien_so_xe):
        self.danh_sach_xe = [xe for xe in self.danh_sach_xe if xe.bien_so_xe != bien_so_xe]

    def tinh_gia_xe(self):
        # Tính tổng chi phí cho mỗi xe
        for xe in self.danh_sach_xe:
            xe.gia_gui = self.money_time.tinh_gia(xe.loai_xe, xe.thoi_gian)
    
    def xuat_thong_tin_xe_tren_20k(self):
        # Xuất thông tin xe có giá gửi trên 20k
        xe_tren_20k = [xe for xe in self.danh_sach_xe if xe.gia_gui > 20]
        return xe_tren_20k
    
    def ghi_du_lieu_vao_excel(self, filename="data_xe.xlsx"):
        # Ghi dữ liệu xe vào file Excel
        data = []
        for xe in self.danh_sach_xe:
            data.append([xe.loai_xe, xe.chu_xe, xe.thoi_gian, xe.bien_so_xe, xe.gia_gui])
        df = pd.DataFrame(data, columns=["Loại xe", "Chủ xe", "Thời gian gửi", "Biển số xe", "Giá gửi"])
        df.to_excel(filename, index=False)

# Chạy thử chương trình
ql_xe = QuanLyXe()

# Thêm một số xe vào danh sách
ql_xe.them_xe(Info_Xe("Xe đạp", "Nguyễn Văn A", 10, "29A-12345"))
ql_xe.them_xe(Info_Xe("Xe máy", "Trần Thị B", 3, "29B-67890"))
ql_xe.them_xe(Info_Xe("Xe điện", "Lê Minh C", 7, "29C-11223"))
ql_xe.them_xe(Info_Xe("Ô tô", "Hoàng Vũ D", 2, "29D-44556"))

# Tính toán giá gửi xe
ql_xe.tinh_gia_xe()
# In thông tin những xe có giá trên 20k
xe_tren_20k = ql_xe.xuat_thong_tin_xe_tren_20k()
print("Thông tin những xe có giá gửi trên 20k:")
for xe in xe_tren_20k:
    print(xe)

# Ghi dữ liệu vào file Excel
ql_xe.ghi_du_lieu_vao_excel("data_xe.xlsx")

Thông tin những xe có giá gửi trên 20k:
Loại xe: Xe điện, Chủ xe: Lê Minh C, Thời gian gửi xe: 7 giờ, Biển số: 29C-11223
